# Capstone Project

Author: Peter Strand

---

### Importing Required Libraries


In [2]:
import pandas as pd
import datetime
import math
import os

Defining the filename and corresponding sheet names to be used throughout document

In [3]:
filename = f"{os.getcwd()}/data/I70_Traffic.xlsx"

sheetname_counts = 'I70_counts'
sheetname_stations = 'I70A_stations'
sheetname_grades = 'Highway_Grades_in_2021'
sheetname_mile_marker = 'mile_marker'
sheetname_curves = 'Highway_Curves_in_2021'
sheetname_pop = 'Population_County'

output_folder = 'output'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

## Defining the initial Traffic Count dataset

In [4]:
traffic = pd.read_excel(filename,sheetname_counts)

#### Data Cleaning

---

In [5]:
# traffic.info()
# traffic.shape
# traffic.dtypes
traffic

,COUNTLOCATION,ROUTE,COUNTSTATIONID,DIRECTION,COUNTDATE,CALYR,DAYOFWEEK,HOUR0,HOUR1,HOUR2,...,HOUR14,HOUR15,HOUR16,HOUR17,HOUR18,HOUR19,HOUR20,HOUR21,HOUR22,HOUR23
0,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180822,2018,4,134,85,78,...,966,1196,1243,1359,993,670,506,348,292,186
1,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180907,2018,6,156,103,99,...,957,1167,1341,1380,1050,759,625,483,409,259
2,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180906,2018,5,133,96,86,...,988,1149,1311,1305,1013,742,533,405,288,192
3,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180829,2018,4,137,85,79,...,901,1133,1360,1349,979,657,585,409,333,172
4,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180828,2018,3,97,105,79,...,873,1093,1266,1355,959,717,578,401,251,161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55066,"ON I-70 W/O HAVANA ST, DENVER",070A,103078,S,20230126,2023,5,1048,693,650,...,7655,8095,8043,7370,6004,5325,4245,3487,2340,2129
55067,"ON I-70 W/O HAVANA ST, DENVER",070A,103078,S,20230125,2023,4,911,621,591,...,7310,7844,7698,7022,5829,4901,3543,2988,2343,1737
55068,"ON I-70 W/O HAVANA ST, DENVER",070A,103078,S,20230124,2023,3,965,767,641,...,7115,8306,7676,7146,5811,4402,3256,2506,2101,1327
55069,"ON I-70 W/O HAVANA ST, DENVER",070A,103078,S,20230123,2023,2,1338,740,661,...,7171,7879,7567,6832,5369,4437,3198,2617,2098,1478


#### Initial Findings
1. Looks like there is a row for each day and the traffic counts every hour, resulting in many columns. 
2. The `COUNTDATE` is a float and not in correct date format
3. There are 55071 rows with 31 columns

In [5]:
traffic.isna().sum(); 

#### Null Findings
1. There are zero nulls in this dataset

---

## Data Manipulation

For my analysis I would like to be able to look at traffic changes over time so I will need to redistribute each hourly value on a new row of data.


In [6]:
id_vars_list_traf = []

for i in range(len(traffic.columns)-7): # Setting range for the melt. Not including the first 7 columns so that all key infomation is stored
    id_vars_list_traf.append(list(traffic.columns[:-i-1]))  # Creating a List of Lists where each subsequent list has the last item removed so that as the melt iterates it is always melting the last column and value
    
var_name_list_traf = traffic.columns[-24:]  # Capturing just the final 24 columns which contain the hourly counts


# Creating a new dataframe that has a new row for every hourly count
melted_dfs = []
for i in range(len(id_vars_list_traf)): # Melting using the lists above that iterates over all 24 hour counts
    melted_traffic = traffic.melt(id_vars = id_vars_list_traf[i], 
                                  var_name = var_name_list_traf[i],
                                  value_name='Count')
    melted_dfs.append(melted_traffic)

### Confirming the new dataset


In [7]:
# melted_traffic.isna().sum()
melted_traffic.head()
# melted_traffic.shape

,COUNTLOCATION,ROUTE,COUNTSTATIONID,DIRECTION,COUNTDATE,CALYR,DAYOFWEEK,HOUR23,Count
0,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180822,2018,4,HOUR0,134
1,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180907,2018,6,HOUR0,156
2,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180906,2018,5,HOUR0,133
3,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180829,2018,4,HOUR0,137
4,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",070A,2,P,20180828,2018,3,HOUR0,97


- The new dataset now has 1,321,704 rows and 8 columns.
- This is the expected amount since the orginal rows (55,073) * hours (24) = new rows (1,321,704)

---
#### Additional Data Cleaning

- Adding the time and converting the date to a date format

In [8]:
melted_traffic['COUNTDATE_TIME'] = melted_traffic['COUNTDATE'].astype(str) + melted_traffic['HOUR23']

Struggled with importing the date correctly into Tableau so kept mulitple types of the `COUNTDATE`
- Created a new column to add the time to the existing date
- Converted the exisiting date float to a date format
- Renamed the `HOUR23` column and dropped the `HOUR` from the value

In [9]:
def convertdatetime(row):
    date_str = row['COUNTDATE_TIME']
    date_obj = datetime.datetime.strptime(date_str, '%Y%m%dHOUR%H')
    return date_obj

melted_traffic['COUNTDATE_TIME'] = melted_traffic.apply(convertdatetime, axis = 1)

In [10]:
def convertdate(row):
    date_int = row['COUNTDATE']
    date_str = str(date_int)
    date = datetime.datetime.strptime(date_str, '%Y%m%d').date()
    return date

melted_traffic['COUNTDATE'] = melted_traffic.apply(convertdate, axis = 1)

In [11]:
melted_traffic = melted_traffic.rename(columns={'HOUR23': 'Hour'})

In [12]:
melted_traffic['Hour'] = melted_traffic['Hour'].map(lambda x: x.lstrip('HOUR'))

Changing the Direction Values
- Switching `P` (Primary) to Eastbound
- Switching `S` (Secondary) to Westbound

In [13]:
melted_traffic.loc[melted_traffic['DIRECTION'] == 'P', 'DIRECTION'] = 'East'
melted_traffic.loc[melted_traffic['DIRECTION'] == 'S', 'DIRECTION'] = 'West'

##### Export to CSV


In [14]:
melted_traffic.to_csv(f'{output_folder}/melted_traffic.csv', index=False)

-----

## Defining the Station dataset

In [15]:
stations = pd.read_excel(filename,sheetname_stations)

Checking the data

In [16]:
# stations.info()
# stations.shape
# stations.dtypes
# stations.duplicated().sum()
stations.duplicated(subset = 'StationID').sum()

0

In [17]:
stations

,StationID,County,Functional Class,Rural Urban,Location,Directions,Latitude,Longitude
0,2,Adams,(1) Interstate,R,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",EB/WB,39.739875,-104.671758
1,11,Eagle,(1) Interstate,R,"ON I-70 SE/O SH 131, WOLCOTT",EB/WB,39.690678,-106.663292
2,14,Lincoln,(1) Interstate,R,"ON I-70 E/O SH 24 & SH 40, E/O LIMON",EB/WB,39.266509,-103.605098
3,104,Mesa,(1) Interstate,R,"ON I-70 W/O SH 6, MACK",EB/WB,39.218190,-108.877344
4,105,Garfield,(1) Interstate,U,"ON I-70 E/O SH 82, GLENWOOD SPRINGS",EB/WB,39.561943,-107.298682
5,106,Summit,(1) Interstate,R,"ON I-70 NE/O SH 9, BLUE RIVER PKWY, AT EISENHO...",EB/WB,39.678705,-105.935154
6,107,Jefferson,(1) Interstate,U,"ON I-70 E/O MT VERNON COUNTRY CLUB RD, GENESEE",EB/WB,39.708895,-105.287368
7,108,Arapahoe,(1) Interstate,R,"ON I-70 W/O SH 36, BYERS",EB/WB,39.721943,-104.247422
8,109,Kit Carson,(1) Interstate,R,ON I-70 NE/O SH 70 BURLINGTON SPUR,EB/WB,39.306858,-102.247197
9,119,Summit,(1) Interstate,R,"ON I-70 W/O SH 91, COPPER MOUNTAIN",EB/WB,39.503080,-106.150063


Dataset Details
- 16 Rows and 8 Columns
- Zero Duplicated Rows
- Zero Null Values
- All unique 'StationID' numbers

Based on these details the data is already clean and is ready for export

##### Export to CSV


In [18]:
stations.to_csv(f'{output_folder}/stations.csv', index=False)

# melted_traffic.to_csv(f'{output_folder}/{'melted_traffic.csv'}', index=False)

----

## Defining the Highway Grade Dataset

In [19]:
grades = pd.read_excel(filename,sheetname_grades)

Examining the data

In [20]:
# grades.info()
# grades.shape
# grades.dtypes
# grades.duplicated().sum()
grades.isna().sum()

ROUTE                 0
REFPT                 0
ENDREFPT              0
RUNLENGTH_FROM        0
RUNLENGTH_TO          0
SEG_LENGTH            0
GRADE_UPDATEYR        0
PRIGRADE              0
PRIGRADECLASS         0
SECGRADE          61830
SECGRADECLASS     61830
OBJECTID              0
dtype: int64

#### The Grades dataset
- 87,627 Rows with 12 columns
- There are no duplicated rows
- The only columns that contain nulls are `SECGRADE` and `SECGRADECLASS` 
    - Both these columns have the same amount of nulls 

## Data Manipulation

- For my analysis am only interested in a specific route, so I will create a mask to segment only that data.
- Change the `PRI` and `SEC` prefixes to East and West
- Add a Mile Marker value for joining purposes based off the `RUNLENGTH_FROM` column


In [21]:
grades_70 = grades[grades['ROUTE'] == '070A']

In [22]:
grades_70 = grades_70.rename(columns={'PRIGRADE': 'EAST_GRADE', 'PRIGRADECLASS': 'EAST_GRADECLASS','SECGRADE': 'WEST_GRADE', 'SECGRADECLASS': 'WEST_GRADECLASS'})

In [23]:
grades_70['mile_marker'] = grades_70['RUNLENGTH_FROM']

In [24]:
# grades_70.info()
grades_70.shape
# grades_70.dtypes
# grades_70.duplicated().sum()
# grades_70.isna().sum()

(4405, 13)

#### The Grades_70 dataset
- 4,405 Rows with 13 columns
- There are no duplicated rows
- The are not any null values


In [25]:
grades_70.sort_values('RUNLENGTH_FROM')

,ROUTE,REFPT,ENDREFPT,RUNLENGTH_FROM,RUNLENGTH_TO,SEG_LENGTH,GRADE_UPDATEYR,EAST_GRADE,EAST_GRADECLASS,WEST_GRADE,WEST_GRADECLASS,OBJECTID,mile_marker
38190,070A,0.000,0.101,0.0,0.10,0.10,2018,5.1,D,-5.0,D,38189,0.0
38191,070A,0.101,0.202,0.1,0.20,0.10,2018,5.6,D,-4.0,C,38190,0.1
38192,070A,0.202,0.303,0.2,0.30,0.10,2018,4.7,D,-3.0,C,38191,0.2
38193,070A,0.303,0.404,0.3,0.40,0.10,2018,4.1,C,-2.6,C,38192,0.3
38194,070A,0.404,0.505,0.4,0.50,0.10,2018,3.8,C,-2.4,B,38193,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42595,070A,449.109,449.209,449.7,449.80,0.10,2018,0.0,A,0.2,A,42589,449.7
42596,070A,449.209,449.309,449.8,449.90,0.10,2018,0.0,A,0.1,A,42590,449.8
42597,070A,449.309,449.409,449.9,450.00,0.10,2018,0.0,A,0.2,A,42591,449.9
42598,070A,449.409,449.509,450.0,450.10,0.10,2018,-0.1,A,0.5,B,42592,450.0


##### Export to CSV


In [26]:
grades_70.to_csv(f'{output_folder}/grades.csv', index=False)

----

## Defining the Mile Marker Dataset

In [27]:
mile_marker = pd.read_excel(filename,sheetname_mile_marker)

Examining the Data

In [28]:
# mile_marker.info()
mile_marker.shape
# mile_marker.dtypes
# mile_marker.duplicated().sum()
# mile_marker.isna().sum()

(11767, 10)

#### The Mile Marker dataset
- 11,767 Rows with 10 columns
- There are 3 duplicated rows
- The only columns that contain nulls are `COMMENTS` and `DATAFILE` 
    - 11,162 and 575 respectively

## Data Manipulation

- The only relevant columns for my analysis are `the_geom`, `DATE_`, `SECTION_`, `PATROL`, `ROUTE`, `ROAD_TYPE`, and `REF_PT`
    - I will drop the other columns
- The coordinates in the 'the_geom' column are nested and need to be extracted into seperate columns
- There are multiple 'REF_PT' that reference the same mile marker
    - I will determine the apppropiate row and drop the additional
- For my analysis am only interested in a specific route, so I will create a mask to segment only that data.


----
Dropping the unnecessary columns

In [29]:
mile_marker.drop(columns = ['COMMENTS', 'DATAFILE','COLLECTOR', 'DATE_'], axis = 1, inplace = True)

Extracting the Latitude and Longitude from the 'the_geom' column and dropping said column

In [30]:
mile_marker['latitude'] = mile_marker['the_geom'].apply(lambda x: float(x.replace('(','').split(' ')[1]))
mile_marker['longitude'] = mile_marker['the_geom'].apply(lambda x: float(x.replace(')','').split(' ')[2]))

In [31]:
mile_marker.drop(columns = 'the_geom', axis = 1, inplace = True)

Defining a Mile Marker Identifier that can be linked to other datasets

In [32]:
mile_marker['mile_marker'] = mile_marker['REF_PT'].round(1)

Filtering the Mile Markers so the dataset only includes those on the Main Highway

In [33]:
mile_marker = mile_marker[mile_marker['ROAD_TYPE'] == 'Main_Hwy']

Dropping all rows except the first instance of a mile marker value
- Upon investigation all duplicated mile marker values shared similar coordinates

In [34]:
mile_marker.drop_duplicates(subset = 'mile_marker', inplace = True)

Filtering the data to only include those mile marker locations along the route we are interested in.

In [35]:
mile_marker_70 = mile_marker[mile_marker['ROUTE'] == '070A']

In [36]:
# mile_marker_70.info()
mile_marker_70.shape
# mile_marker_70.dtypes
# mile_marker_70.duplicated().sum()
# mile_marker_70.isna().sum()

(29, 8)

#### The Mile Marker 70 dataset
- 29 Rows with 8 columns
- There are no duplicated rows
- There are not any null values

##### Export to CSV


In [37]:
mile_marker_70.to_csv(f'{output_folder}/mile_markers.csv', index=False)

---

## Defining the Highway Curve Dataset

In [38]:
curves = pd.read_excel(filename,sheetname_curves)

Examining the Data

In [39]:
# curves.info()
# curves.shape
# curves.dtypes
# curves.duplicated().sum()
curves.isna().sum()

ROUTE                 0
REFPT                 0
ENDREFPT              0
RUNLENGTH_FROM        0
RUNLENGTH_TO          0
SEG_LENGTH            0
CURVE_UPDATEYR        0
PRICURVE              0
PRICURVECLASS         0
SECCURVE          29832
SECCURVECLASS     29832
OBJECTID              0
dtype: int64

#### The Curves dataset
- 29,832 Rows with 12 columns
- There are no duplicated rows
- The are 29,832 null values in `SECCURVE` and `SECCURVECLASS`
    - It seems like curves were only measured in the primary direction 
    - Both columns can be dropped

Dropping unnecessary columns

In [40]:
curves.drop(columns = ['SECCURVE','SECCURVECLASS'], axis = 1, inplace = True)

Creating a marker value to link to other datasets
- The curvature is measured as arclength so for mile marker location I will use the center of the curve
- Rounding to 1 decimal place to match the other mile markers

In [41]:
curves['mile_marker'] = ((curves['RUNLENGTH_FROM'] + curves['RUNLENGTH_TO'])/2).round(1)

Filtering out only the route I am interested in

In [42]:
curves = curves[curves['ROUTE']=='070A']

There are 128 duplicate marker IDs
   - After some examining it seems like some curves were designated twice
       - But onc occurance has a `PRICURVE` value of -1 which denotes a straightline
           - Dropped all duplicates with a `PRICURVE` value of -1

In [43]:
curves = curves.iloc[abs(curves['PRICURVE']).argsort()].drop_duplicates(subset = 'mile_marker', keep = 'last')

In [44]:
curves = curves.rename(columns={'PRICURVE': 'EAST_CURVE', 'PRICURVECLASS': 'EAST_CURVECLASS'})

##### Export to CSV


In [45]:
curves.to_csv(f'{output_folder}/curves.csv', index=False)

---

## Defining the Population Dataset

In [46]:
pop = pd.read_excel(filename,sheetname_pop)

Examining the Data

In [47]:
# pop.info()
# pop.shape
pop.dtypes
# pop.duplicated().sum()
# pop.isna().sum()

county             object
year                int64
totalPopulation     int64
dtype: object

#### The Population dataset
- 3,904 Rows with 3 columns
- There are no duplicated rows
- The are zero null values

## Data Manipulation

- I am only interested in the counties on the Front Range for my analysis
    - Removing all Counties not on the Front Range
- My traffic data includes the entire years from 2018-2022
    - Remove all year outside this range

In [48]:
# keep_counties = ['Broomfield','Denver','Arapahoe', 'Boulder', 'Jefferson','Adams', 'Douglas']
# pop_keep = pop['county'].isin(keep_counties)
# pop = pop[pop_keep]

In [49]:
keep_years = list(range(2000,2023))
pop_years = pop['year'].isin(keep_years)
pop = pop[pop_years]

##### Export to CSV

In [50]:
pop.to_csv(f'{output_folder}/population.csv', index=False)

----

## Defining the Annual Traffic Dataset
 

Importing all datasets from 2011 to 2021 and unioning them into one complete dataset

In [51]:
annual_traf = [] 

for i in range(2011, 2022):
    df = pd.read_excel(f"{os.getcwd()}/data/annual_traffic/CDOT_Traffic{i}.xlsx",header = 0)
    annual_traf.append(df)
     
traffic_all = pd.concat(annual_traf, ignore_index=True)

/Users/peter/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Defining the specific columns that are needed
- When examining the data throughout the years it seems like the column names changed 

In [52]:
keep_columns = ['COUNTSTATI', 'AADT', 'CALYR', 'COUNTSTATIONID', 'REFPT']
traffic_all = traffic_all.loc[:, keep_columns]

Filling all null values in `COUNTSTATIONID` and setting as an interger before dropping the `COUNTSTATI` column

In [53]:
traffic_all['COUNTSTATIONID'].fillna(traffic_all['COUNTSTATI'], inplace = True)
traffic_all['COUNTSTATIONID'] = traffic_all['COUNTSTATIONID'].astype(int)
traffic_all.drop(columns = 'COUNTSTATI', inplace= True)

Joining the `traffic_all` dataframe with the `stations` dataframe and dropped unnecessary columns

In [54]:
traffic_merged = pd.merge(stations,traffic_all, left_on = 'StationID',right_on ='COUNTSTATIONID', how = 'left')
traffic_merged.drop(columns = ['Functional Class','Rural Urban','COUNTSTATIONID'], inplace = True)

Separating out the unique Station IDs in order to build a dictionary of consecutive stations 

In [55]:
traffic_merged_unique = traffic_merged.drop_duplicates(subset=['StationID'], keep='first')
traffic_merged_unique = traffic_merged_unique.sort_values('REFPT')
traffic_merged_unique.loc[:, 'next_station'] = traffic_merged_unique['StationID'].shift(1)

In [56]:
station_dict = traffic_merged_unique.drop(columns = [ 'County', 'Location', 'Directions', 'Latitude','Longitude', 'AADT', 'CALYR', 'REFPT'])
station_dict.set_index('StationID')['next_station'].to_dict()

{104: nan,
 103011: 104.0,
 105: 103011.0,
 11: 105.0,
 126: 11.0,
 119: 126.0,
 106: 119.0,
 120: 106.0,
 107: 120.0,
 510: 107.0,
 511: 510.0,
 103078: 511.0,
 2: 103078.0,
 108: 2.0,
 14: 108.0,
 109: 14.0}

In [58]:
dict_1= {104:None, 103011:104,105:103011,11:105,126:11,119:126,106:119,120:106,107:120,510:107,511:510,103078:511,2:103078,108:2,14:108,109:14}

In [59]:
traffic_merged['to_station'] = traffic_merged['StationID'].apply(lambda x: dict_1[x])
traffic_merged.dropna(subset = ['to_station'], axis = 0,inplace = True)

In [60]:
traffic_merged = pd.merge(traffic_merged,stations, right_on = 'StationID',left_on ='to_station', how = 'left', suffixes=('_start', '_end'))
traffic_merged;

##### Export to CSV

In [61]:
traffic_merged.to_csv(f'{output_folder}/traffic_merged.csv', index=False)

----